In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [5]:
A = [1,2,3,4,5,6]
B = [13, 21, 34]
A.extend(B)

In [6]:
A

[1, 2, 3, 4, 5, 6, 13, 21, 34]

In [7]:
np.identity(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [8]:
df = pd.read_csv('fuel_ferc1.csv')
df.head()

,record_id,utility_id_ferc1,report_year,plant_name_ferc1,fuel_type_code_pudl,fuel_unit,fuel_qty_burned,fuel_mmbtu_per_unit,fuel_cost_per_unit_burned,fuel_cost_per_unit_delivered,fuel_cost_per_mmbtu
0,f1_fuel_1994_12_1_0_7,1,1994,rockport,coal,ton,5377489.0,16.590,18.59,18.53,1.121
1,f1_fuel_1994_12_1_0_10,1,1994,rockport total plant,coal,ton,10486945.0,16.592,18.58,18.53,1.120
2,f1_fuel_1994_12_2_0_1,2,1994,gorgas,coal,ton,2978683.0,24.130,39.72,38.12,1.650
3,f1_fuel_1994_12_2_0_7,2,1994,barry,coal,ton,3739484.0,23.950,47.21,45.99,1.970
4,f1_fuel_1994_12_2_0_10,2,1994,chickasaw,gas,mcf,40533.0,1.000,2.77,2.77,2.570


In [10]:
# which of the following fuel type code has the lowest average fuel cost per unit burned?
df.groupby('fuel_type_code_pudl')['fuel_cost_per_unit_burned'].mean().sort_values()

fuel_type_code_pudl
gas          13.659397
other        18.253856
waste        19.518122
coal         67.421830
oil         168.877086
nuclear    4955.157002
Name: fuel_cost_per_unit_burned, dtype: float64

In [11]:
# What is the standard deviation and 75th percentile of the measure of energy per unit (Fuel_mmbtu_per_unit) in two decimal places?
df['fuel_mmbtu_per_unit'].describe()

count    29523.000000
mean         8.492111
std         10.600220
min          0.000001
25%          1.024000
50%          5.762694
75%         17.006000
max        341.260000
Name: fuel_mmbtu_per_unit, dtype: float64

In [14]:
# What is the skewness and kurtosis for the fuel quantity burned in two decimal places?
skew_fuel_qty_burned = df['fuel_qty_burned'].skew().round(2)
kurtosis_fuel_qty_burned = df['fuel_qty_burned'].kurtosis().round(2)
print(f'SKew: {skew_fuel_qty_burned} and Kurtosis {kurtosis_fuel_qty_burned}')

SKew: 15.85 and Kurtosis 651.37


In [15]:
# the data will need alot of cleaning maybe not so much 
# let get started

# Function to calculate missing values by column
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [16]:
missing_values_table(df)

Your selected dataframe has 11 columns.
There are 1 columns that have missing values.


,Missing Values,% of Total Values
fuel_unit,180,0.6


In [17]:
# The feature with missing values falls under what category? What missing value imputation technique would you use?
df['fuel_unit'].dtype

dtype('O')

In [23]:
# Question 8
# Which of the features has the second and third lowest correlation with the Fuel Cost Per Unit Burned?
df.corr()['fuel_cost_per_unit_burned'].sort_values()

utility_id_ferc1               -0.037863
fuel_qty_burned                -0.018535
fuel_mmbtu_per_unit            -0.010034
fuel_cost_per_mmbtu            -0.000437
fuel_cost_per_unit_delivered    0.011007
report_year                     0.013599
fuel_cost_per_unit_burned       1.000000
Name: fuel_cost_per_unit_burned, dtype: float64

In [24]:
bool(-0.037863 > -0.018535)

False

In [25]:
# For the fuel type coal, what is the percentage change in the fuel cost per unit burned in 1998 compared to 1994?

col_df = df[df['fuel_type_code_pudl'] == 'coal']
col_df.shape

(8547, 11)

In [31]:
# Question 9
((11902.597 - 14984.572) / (14984.572)) * 100

-20.567654518260518

In [37]:
# Question 10
# Which year has the highest average fuel cost per unit delivered?
col_df.groupby('report_year')['fuel_cost_per_unit_delivered'].mean().sort_values(ascending=False)

report_year
2018    1204.041209
1998     716.015659
2010     184.509668
2000     173.192467
1994     124.663939
2013      50.578059
2012      50.378920
2011      49.683282
2014      48.970075
2009      45.534962
2015      45.162299
2008      42.571168
2016      42.345494
2017      40.528471
2001      39.145055
2007      36.397688
2006      35.366483
1999      34.763268
2005      32.921988
1995      28.850834
2004      28.491137
1996      28.045432
1997      27.119043
2003      26.722605
2002      26.107316
Name: fuel_cost_per_unit_delivered, dtype: float64